### Setup

In [1]:
using Pkg

# Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
# Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

    Updating registry at `C:\Users\bergermann\.julia\registries\General.toml`


    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`


    Updating git-repo `https://github.com/bergermann/Dragoon.jl.git`


  No Changes to `C:\Users\bergermann\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\bergermann\.julia\environments\v1.9\Manifest.toml`


In [2]:
using Dragoon
using Plots
# using Dates

In [3]:
#number of discs in the booster
n = 20

#initial disc configuration
#findpeak tries to find an equidistant configuration with a peak at f
initdist = findpeak(22.025e9,n)

#generate frequencies for calculation and for plotting
freqs = genFreqs(22.025e9,50e6; length=10) #optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; length=1000)

#initialize physical properties of the booster
booster = AnalyticalBooster(initdist)
# booster.timestamp = DateTime(0)
#=
Booster(
    pos,              disc positions
    ndisk,            disc number
    thickness,        disc thickness
    epsilon,          disc epsilon
    vmotor,           motor speed
    maxlength,        maximum allowed booster length (not implemented yet)
    timestamp,        booster operation time
    summedtraveltime, summed motor operation time
    codetimestamp     code runtime
)
=#

#initialize storage of measurements
hist = initHist(booster,10000,freqs,ObjAnalytical)
#                           ^ length, should be at least the larger of (n^2, search steps)
;

Objective Value: -243.3, Timestamp: 0000-01-01T00:00:00


### Optimizer

In [4]:
#=
how to use

T = collect(range(100; length=1001,stop=0))

trace = simulatedAnnealing(booster,hist,freqs,
            T,100e-6,
            ObjAnalytical,
            UnstuckDont;
            maxiter=Int(1e6),
            showtrace=true,
            showevery=100,
            unstuckisiter=true,
            traceevery=100,
            resettimer=true)

returns trace, access with
trace[iteration].x      disc position
                .obj    objective value
                .g      gradient
                .h      hessian
                .t      timestamp
                .T      summed travel time
=#

In [5]:
T = collect(range(100; length=1001,stop=0))

trace = simulatedAnnealing(booster,hist,freqs,
            T,100e-6,
            ObjAnalytical,
            UnstuckDont;
            maxiter=Int(1e6),
            showtrace=true,
            showevery=100,
            unstuckisiter=true,
            traceevery=1,
            resettimer=true)
;

BoundsError: BoundsError: attempt to access 1001-element Vector{Float64} at index [0]

In [6]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

20-element Vector{Float64}:
 0.007209715011297336
 0.01541943002259467
 0.023629145033892004
 0.03183886004518934
 0.04004857505648668
 0.04825829006778401
 0.05646800507908135
 0.06467772009037868
 0.07288743510167603
 0.08109715011297336
 0.08930686512427068
 0.097516580135568
 0.10572629514686534
 0.11393601015816267
 0.12214572516946
 0.13035544018075734
 0.13856515519205465
 0.14677487020335206
 0.15498458521464936
 0.1631943002259467

In [7]:
#get handy output
analyse(hist,trace[1:end],freqsplot; freqs=freqs,div=5,scale=1e9,ylim=[-0.05e4,3e4])

#savefig(plt[i],"cool_epic_result_wow.svg")

UndefVarError: UndefVarError: `trace` not defined